In [2]:
import os
import sys
sys.path.insert(1, os.path.abspath('..'))
import django_initializer

In [5]:
from heros.configuration.models import ConfigDocument, Package, PackageVersion
from heros.configuration.serializers import ConfigDocumentSerializer
from git import Repo
from os import path
import os
import shutil
import json

document_types = ['context','object','method']
repo_base_path = path.join("..","repos")

class PackageRepository:
    def __init__(self, package, version=None):
        if not package.remote:
            raise Exception("Remote repository not updated in package")
        self.package = package
        self.version = version
        self.repo_path = path.join(repo_base_path, package.code)
        if os.path.isdir(self.repo_path):
            shutil.rmtree(self.repo_path)
        self.repo = Repo.clone_from(package.remote,self.repo_path)

    def ensure_version(self):
        if not self.version:
            raise Exception("Version required")

    def checkout_version(self):
        self.ensure_version()
        if self.version.remote_commit == 'initial':
            self.repo.git.checkout('--orphan',self.version.version)
        else: 
            self.repo.git.checkout(self.version.version)

    def write_documents_to_repo(self):
        self.ensure_version()
        for document in self.version.config_documents.all():
            document_git_path = '/'.join([
                'docs', 
                document.document_type, 
                document.code + ".json"])

            document_path = path.join(
                self.repo_path ,
                'docs', 
                document.document_type, 
                document.code + ".json")

            data = ConfigDocumentSerializer(document).data
            os.makedirs(os.path.dirname(document_path), exist_ok=True)
            with open(document_path, "w") as outfile:
                json.dump(data, outfile,indent=4)
            self.repo.index.add(document_git_path)
    def commit(self, message):
        return self.repo.index.commit(message)

version = PackageVersion.objects.get(pk=1)
repo = PackageRepository(version.package,version)
repo.checkout_version()
repo.write_documents_to_repo()
commit = repo.commit("New version")
print(commit.hexsha)



30dd5d98932ca86d7e39a555d0b4a7d3910c0915


In [15]:




def pull_version(package_version_id):
    version = PackageVersion.objects.get(pk=package_version_id)
    repo_path = path.join(repo_base_path,version.package.code)

    if version.package.remote:
        repo = Repo.clone_from(version.package.remote,repo_path)
        if version.remote_commit == 'initial':
            repo.git.checkout('--orphan',version.version)
        else: 
            repo.git.checkout(version.version)
        for document in version.config_documents.all():
            document_path = path.join(repo_path,document.document_type,document.code + ".json")
            data = ConfigDocumentSerializer(document).data
            os.makedirs(os.path.dirname(document_path), exist_ok=True)
            with open(document_path, "w") as outfile:
                json.dump(data, outfile,indent=4)

pull_version(1)        